In [1]:
import env
import os
import acquire
import numpy as np
import pandas as pd
from scipy import stats
from pydataset import data
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt

### 2. Given the following confusion matrix, evaluate (by hand) the model's performance.
|               | pred dog   | pred cat   |
|:------------  |-----------:|-----------:|
| actual dog    |         46 |         7  |
| actual cat    |         13 |         34 |
- In the context of this problem, what is a false positive?
- In the context of this problem, what is a false negative?
- How would you describe this model?

In [2]:
data= {'actual_dog':[46,7],'actual_cat':[13,34]}
catdog = pd.DataFrame(data=data,index=['pred_dog','pred_cat'])

# false pos true neg
# false neg true pos

totalp = 80
totaln = 20
total = 100

tp = 34
fp = 46
tn = 13
fn = 7
catdog

,actual_dog,actual_cat
pred_dog,46,13
pred_cat,7,34


In [3]:
def eval_metrics(tp,tn,fp,fn):
    '''Input:
    True positive(tp),
    True negative (tn),
    False positive (fp),
    False negative (fn)

    Reminder:
    false pos true neg
    false neg true pos
    '''
    accuracy = (tp + tn) / (tp + fp + fn + tn)
    precision = tp / (tp+fp)
    recall = tp / (tp+fn)
    return print(f'''Accuracy is: {accuracy}\nPrecision is: {precision}\nRecall is: {recall}''')

### 2A.
- A false positive would be the 46 value the true positive is 34
- A true negative would be 7 value the false negatives would be 13
- This model is has 47% accuracy, most of it's positives were false positives only had a precision score of 42.5%

### 3. You are working as a datascientist working for Codeup Cody Creator (C3 for short), a rubber-duck manufacturing plant.

Unfortunately, some of the rubber ducks that are produced will have defects. Your team has built several models that try to predict those defects, and the data from their predictions can be found here.

Use the predictions dataset and pandas to help answer the following questions:

- An internal team wants to investigate the cause of the manufacturing defects. They tell you that they want to identify as many of the ducks that have a defect as possible. Which evaluation metric would be appropriate here? Which model would be the best fit for this use case?
- Recently several stories in the local news have come out highlighting customers who received a rubber duck with a defect, and portraying C3 in a bad light. The PR team has decided to launch a program that gives customers with a defective duck a vacation to Hawaii. They need you to predict which ducks will have defects, but tell you the really don't want to accidentally give out a vacation package when the duck really doesn't have a defect. Which evaluation metric would be appropriate here? Which model would be the best fit for this use case?


In [4]:
c3 = 'c3.csv'
if os.path.exists(c3):
    df = pd.read_csv(c3)

c3 = df

c3

,actual,model1,model2,model3
0,No Defect,No Defect,Defect,No Defect
1,No Defect,No Defect,Defect,Defect
2,No Defect,No Defect,Defect,No Defect
3,No Defect,Defect,Defect,Defect
4,No Defect,No Defect,Defect,No Defect
...,...,...,...,...
195,No Defect,No Defect,Defect,Defect
196,Defect,Defect,No Defect,No Defect
197,No Defect,No Defect,No Defect,No Defect
198,No Defect,No Defect,Defect,Defect


In [5]:
#c3.replace(to_replace='No Defect','')

In [6]:
# model 1 crosstab
pd.crosstab(c3.actual,c3.model1)

model1,Defect,No Defect
actual,,
Defect,8,8
No Defect,2,182


In [7]:
# model 1
c = confusion_matrix(c3.actual, c3.model1, labels=('Defect','No Defect'))
print(eval_metrics(tp=182,tn=8,fp=8,fn=2))
c

Accuracy is: 0.95
Precision is: 0.9578947368421052
Recall is: 0.9891304347826086
None


array([[  8,   8],
       [  2, 182]])

In [8]:
# model 2 crosstab
pd.crosstab(c3.actual,c3.model2)

model2,Defect,No Defect
actual,,
Defect,9,7
No Defect,81,103


In [9]:
# model 2
c= confusion_matrix(c3.actual, c3.model2, labels=('Defect','No Defect'))
eval_metrics(tp=103,tn=7,fp=9,fn=81)
c

Accuracy is: 0.55
Precision is: 0.9196428571428571
Recall is: 0.5597826086956522


array([[  9,   7],
       [ 81, 103]])

In [10]:
# model 3 crosstab
pd.crosstab(c3.actual,c3.model3)

model3,Defect,No Defect
actual,,
Defect,13,3
No Defect,86,98


In [11]:
# model 3
c = confusion_matrix(c3.actual, c3.model3, labels=('Defect','No Defect'))
eval_metrics(tp=98,tn=3,fp=13,fn=86)
c

Accuracy is: 0.505
Precision is: 0.8828828828828829
Recall is: 0.532608695652174


array([[13,  3],
       [86, 98]])

In [12]:
accuracy_score(c3.actual,c3.model3)

0.555

### 3A.
Model 1 would be the best model with these evaluation metrics
- Accuracy is: 0.95
- Precision is: 0.9578947368421052
- Recall is: 0.9891304347826086

Model 1 would still be best.

### 4. You are working as a data scientist for Gives You Paws ™, a subscription based service that shows you cute pictures of dogs or cats (or both for an additional fee).

At Gives You Paws, anyone can upload pictures of their cats or dogs. The photos are then put through a two step process. First an automated algorithm tags pictures as either a cat or a dog (Phase I). Next, the photos that have been initially identified are put through another round of review, possibly with some human oversight, before being presented to the users (Phase II).

Several models have already been developed with the data, and you can find their results here.

Given this dataset, use pandas to create a baseline model (i.e. a model that just predicts the most common class) and answer the following questions:

- In terms of accuracy, how do the various models compare to the baseline model? Are any of the models better than the baseline?
- Suppose you are working on a team that solely deals with dog pictures. Which of these models would you recommend?
- Suppose you are working on a team that solely deals with cat pictures. Which of these models would you recommend?

In [16]:
paws = 'gives_you_paws.csv'
if os.path.exists(paws):
    df = pd.read_csv(paws)

paws = df
paws

,actual,model1,model2,model3,model4
0,cat,cat,dog,cat,dog
1,dog,dog,cat,cat,dog
2,dog,cat,cat,cat,dog
3,dog,dog,dog,cat,dog
4,cat,cat,cat,dog,dog
...,...,...,...,...,...
4995,dog,dog,dog,dog,dog
4996,dog,dog,cat,cat,dog
4997,dog,cat,cat,dog,dog
4998,cat,cat,cat,cat,dog


In [17]:
# model 1 crosstab
pd.crosstab(paws.actual,paws.model1)

model1,cat,dog
actual,,
cat,1423,323
dog,640,2614


In [19]:
# model 1
c = confusion_matrix(paws.actual, paws.model1, labels=('cat','dog'))
print(eval_metrics(tp=2614,tn=640,fp=1423,fn=323))
c

Accuracy is: 0.6508
Precision is: 0.6475105276195194
Recall is: 0.8900238338440586
None


array([[1423,  323],
       [ 640, 2614]])

In [28]:
# model 2 crosstab
pd.crosstab(paws.actual,paws.model2)

model2,cat,dog
actual,,
cat,1555,191
dog,1657,1597


In [27]:
# model 2
c = confusion_matrix(paws.actual, paws.model2, labels=('cat','dog'))
print(eval_metrics(tp=1597,tn=1657,fp=1555,fn=191))
c

Accuracy is: 0.6508
Precision is: 0.5066624365482234
Recall is: 0.8931767337807607
None


array([[1555,  191],
       [1657, 1597]])

In [21]:
# model 3 crosstab
pd.crosstab(paws.actual,paws.model3)

model3,cat,dog
actual,,
cat,893,853
dog,1599,1655


In [26]:
# model 3
c = confusion_matrix(paws.actual, paws.model3, labels=('cat','dog'))
print(eval_metrics(tp=3110,tn=144,fp=603,fn=1143))
c

Accuracy is: 0.6508
Precision is: 0.8375976299488285
Recall is: 0.7312485304490948
None


array([[ 893,  853],
       [1599, 1655]])

In [22]:
# model 4 crosstab
pd.crosstab(paws.actual,paws.model4)

model4,cat,dog
actual,,
cat,603,1143
dog,144,3110


In [25]:
# model 4
c = confusion_matrix(paws.actual, paws.model4, labels=('cat','dog'))
print(eval_metrics(tp=3110,tn=144,fp=603,fn=1143))
c

Accuracy is: 0.6508
Precision is: 0.8375976299488285
Recall is: 0.7312485304490948
None


array([[ 603, 1143],
       [ 144, 3110]])